
# 🛰 Simple Satellite Tracker
A streamlined notebook for tracking satellites with realistic 3D visualization
---



Satellite tracker

In [1]:
# Environment Setup and Library Installation
!pip install poliastro requests plotly matplotlib astropy

import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import requests
from datetime import datetime
from astropy import units as u

# Fetch ISS TLE data
def fetch_iss_tle():
    """Fetch current ISS TLE data from CelesTrak"""
    url = "https://celestrak.org/NORAD/elements/gp.php?CATNR=25544&FORMAT=tle"
    try:
        response = requests.get(url, timeout=10)
        lines = response.text.strip().split('\n')
        return lines[0].strip(), lines[1].strip(), lines[2].strip()
    except:
        # Backup TLE data
        return ("ISS (ZARYA)",
                "1 25544U 98067A   24250.54267857  .00016717  00000+0  28940-3 0  9992",
                "2 25544  51.6416 339.3960 0007417  68.6416 291.5736 15.49489315472789")

name, line1, line2 = fetch_iss_tle()
print(f"🚀 Setup complete! Tracking: {name}")
print(f"⏰ Current time: {datetime.now().strftime('%H:%M:%S')}")

Requested poliastro from https://files.pythonhosted.org/packages/1c/ce/b2cf237afeacddd856bb3ae524c44b8aec62e14c13d137283122fd0b5099/poliastro-0.12.0-py3-none-any.whl has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    astropy (<4.*,>=3.1)
             ~~~^
Please use pip<24.1 if you need to use this version.
Requested poliastro from https://files.pythonhosted.org/packages/f7/9a/934e863eee7acca4648b3570085da982cde69969527b9f4d7a0445f16789/poliastro-0.11.1-py3-none-any.whl has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    astropy (<4.*,>=3.0)
             ~~~^
Please use pip<24.1 if you need to use this version.
Requested poliastro from https://files.pythonhosted.org/packages/31/7d/55cfd3a348ed5575d0468e26c65c35295fc743c28598ba790561e065a263/poliastro-0.11.0-py3-none-any.whl has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    astropy (<4.*,>=3.0)
             ~~~^
Please use pip<24.1 if you need to u

In [2]:
# TLE Parsing and Position Calculation
def parse_tle(line1, line2):
    """Extract key orbital elements from TLE"""
    inclination = float(line2[8:16])      # degrees
    eccentricity = float("0." + line2[26:33])
    mean_motion = float(line2[52:63])     # revolutions per day
    period_minutes = 24 * 60 / mean_motion

    return {
        'inclination': inclination,
        'eccentricity': eccentricity,
        'mean_motion': mean_motion,
        'period': period_minutes
    }

def calculate_iss_position(orbital_data, time_offset=0):
    """Calculate ISS position based on current time"""
    orbit_radius = 6371 + 408  # Earth radius + ISS altitude (km)
    inclination_rad = np.radians(orbital_data['inclination'])

    current_time = datetime.now()
    minutes_elapsed = (current_time.hour * 60 + current_time.minute + time_offset) % orbital_data['period']
    orbital_angle = (minutes_elapsed / orbital_data['period']) * 2 * np.pi

    x = orbit_radius * np.cos(orbital_angle)
    y = orbit_radius * np.sin(orbital_angle) * np.cos(inclination_rad)
    z = orbit_radius * np.sin(orbital_angle) * np.sin(inclination_rad)

    return x, y, z

# Parse orbital data
orbital_data = parse_tle(line1, line2)
print(f"🛰 ISS Orbital Data:")
print(f"  Period: {orbital_data['period']:.1f} minutes")
print(f"  Inclination: {orbital_data['inclination']:.1f}°")
print(f"  Orbits/day: {orbital_data['mean_motion']:.1f}")

🛰 ISS Orbital Data:
  Period: 92.9 minutes
  Inclination: 51.6°
  Orbits/day: 15.5


In [3]:
# Create Real-Time Satellite Tracking with Built-in Earth
def create_satellite_tracker(orbital_data):
    """Use Plotly's built-in Earth model for ISS tracking - ISS FIRST!"""

    # Calculate ISS position
    x_iss, y_iss, z_iss = calculate_iss_position(orbital_data)

    # Convert to latitude/longitude for geographic display
    distance = np.sqrt(x_iss**2 + y_iss**2 + z_iss**2)
    lat_iss = np.degrees(np.arcsin(z_iss / distance))
    lon_iss = np.degrees(np.arctan2(y_iss, x_iss))

    # Create figure with built-in Earth
    fig = go.Figure()

    # ADD ISS POSITION FIRST - HUGE AND VISIBLE!
    fig.add_trace(go.Scattergeo(
        lon=[lon_iss],
        lat=[lat_iss],
        mode='markers+text',
        marker=dict(
            size=35,              # MASSIVE marker
            color='#FF0000',      # Bright red
            symbol='circle',
            line=dict(color='yellow', width=5)  # Yellow glow
        ),
        text=['🛰 ISS'],         # Big ISS label
        textfont=dict(size=20, color='white'),
        textposition='top center',
        name='🛰 ISS CURRENT POSITION',
        hovertext=f'🛰 ISS LIVE POSITION<br>⏰ {datetime.now().strftime("%H:%M:%S")}<br>📍 Lat: {lat_iss:.2f}°<br>📍 Lon: {lon_iss:.2f}°<br>🚀 Altitude: 408 km<br>⚡ Speed: 27,600 km/h',
        hoverinfo='text'
    ))

    # Add orbital path SECOND
    orbit_angles = np.linspace(0, 2*np.pi, 150)
    orbit_radius = 6371 + 408
    inclination_rad = np.radians(orbital_data['inclination'])

    x_orbit = orbit_radius * np.cos(orbit_angles)
    y_orbit = orbit_radius * np.sin(orbit_angles) * np.cos(inclination_rad)
    z_orbit = orbit_radius * np.sin(orbit_angles) * np.sin(inclination_rad)

    lat_orbit = np.degrees(np.arcsin(z_orbit / orbit_radius))
    lon_orbit = np.degrees(np.arctan2(y_orbit, x_orbit))

    fig.add_trace(go.Scattergeo(
        lon=lon_orbit,
        lat=lat_orbit,
        mode='lines',
        line=dict(color='cyan', width=2, dash='dot'),
        name='ISS Orbit Path',
        hoverinfo='skip'
    ))

    # DARK SPACE THEME - Professional look
    fig.update_geos(
        projection_type="orthographic",  # 3D globe view
        showland=True,
        landcolor="rgb(100, 80, 60)",     # Dark continents
        showocean=True,
        oceancolor="rgb(20, 40, 80)",     # Dark oceans
        showlakes=True,
        lakecolor="rgb(30, 50, 100)",
        showcountries=True,
        countrycolor="rgb(80, 80, 80)",
        coastlinecolor="rgb(100, 100, 100)",
        bgcolor="rgb(0, 0, 0)",           # Black space
        projection_rotation=dict(lon=lon_iss, lat=lat_iss)  # Center on ISS
    )

    fig.update_layout(
        title=dict(
            text=f"🛰 ISS LIVE: {datetime.now().strftime('%H:%M:%S')} | 📍 {lat_iss:.1f}°, {lon_iss:.1f}°",
            font=dict(color='white', size=18, family='Arial Black')
        ),
        width=900,
        height=700,
        paper_bgcolor='black',
        font=dict(color='white', size=14),
        showlegend=True,
        legend=dict(bgcolor="rgba(0,0,0,0.8)", bordercolor="white", borderwidth=1)
    )

    return fig, lat_iss, lon_iss

# Create tracker - ISS SHOWN FIRST!
tracker_fig, current_lat, current_lon = create_satellite_tracker(orbital_data)
tracker_fig.show()

# Print ISS status prominently
print("🚨 ISS CURRENT STATUS:")
print("=" * 50)
print(f"🛰 POSITION: {current_lat:.3f}°N, {current_lon:.3f}°E")
print(f"⏰ TIME: {datetime.now().strftime('%H:%M:%S')}")
print(f"🚀 ALTITUDE: 408 km")
print(f"⚡ SPEED: 27,600 km/h")
print("=" * 50)

🚨 ISS CURRENT STATUS:
🛰 POSITION: 35.844°N, 34.882°E
⏰ TIME: 06:24:04
🚀 ALTITUDE: 408 km
⚡ SPEED: 27,600 km/h
